In [1]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv(r'D:\Dev\DS_ML\amazon_review_polarity_csv\test.csv', names = ['verdict', 'header', 'review'])

In [15]:
df

,verdict,header,review
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
...,...,...,...
399995,1,Unbelievable- In a Bad Way,We bought this Thomas for our son who is a hug...
399996,1,"Almost Great, Until it Broke...",My son recieved this as a birthday gift 2 mont...
399997,1,Disappointed !!!,"I bought this toy for my son who loves the ""Th..."
399998,2,Classic Jessica Mitford,This is a compilation of a wide range of Mitfo...


In [18]:
data = df[:100][:]

In [20]:
data #try on this

,verdict,header,review
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
...,...,...,...
95,2,The Scarlet Letter a must read for any American,The best book that I have ever been assigned t...
96,1,The Scarlet Letter: A soap opera,"Although this book, is considered a great work..."
97,2,This book is Great!,One might notice that many of the negative rev...
98,1,I had to read it...,I had to read this book for school. I did not ...


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, BertTokenizer

In [24]:
tokeniser = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir=r'E:\models')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2, cache_dir=r'E:\models', device_map="auto")

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`